In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import collections

In [2]:
processed_data_dir = './processed data/'
NO_OF_PATIENTS = 10

In [3]:
files = os.listdir(processed_data_dir)
patients = [np.load(processed_data_dir + s) for s in os.listdir(processed_data_dir)]
image_list = [patients[i][0] for i in range(NO_OF_PATIENTS)]
label_list = [patients[i][1] for i in range(NO_OF_PATIENTS)]
for i in range(NO_OF_PATIENTS):
    image = image_list[i]
    image_list[i] = np.expand_dims(image,1)
    
    
for i in range(NO_OF_PATIENTS):
    print(image_list[i].shape)

(55, 1, 512, 512)
(40, 1, 512, 512)
(35, 1, 512, 512)
(45, 1, 512, 512)
(45, 1, 512, 512)
(30, 1, 512, 512)
(80, 1, 512, 512)
(30, 1, 512, 512)
(30, 1, 512, 512)
(140, 1, 512, 512)


In [4]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=8,kernel_size=5)
        self.relu1 = nn.ReLU()
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=5)
        self.relu2 = nn.ReLU()
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(32*7*7,10)
        
    def forward(self,x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.avgpool1(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.avgpool2(out)
        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        
        return out

In [5]:
learning_rate = 0.1

model = CNNModel()
# if torch.cuda.is_available():
#     model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
num_epochs = 10
iter = 0
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(zip(image_list,label_list)):
        images = torch.from_numpy(images)
        images = images.float()
        if labels == 1:
            labels = torch.tensor([1])
        else:
            labels = torch.tensor([0])
            
#         if torch.cuda.is_available():
#             images = Variable(images.cuda())
#             labels = Variable(labels.cuda())
#         else:
#             images = Variable(images)
#             labels = Variable(labels)

        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1
        if iter%500 == 0:
            correct = 0
            total = 0
            
            for images, labels in test_loader:
                if torch.cuda.is_available():
                    images = Variable(images.cuda())
                else:
                    images = Variable(images)
                outputs = model(images)
                
                _,predicted = torch.max(outputs.data,1)
                total += labels.size(0)
                correct += (predicted.cpu() == labels.cpu()).sum()
            
            accuracy = 100 * correct / total
            print('Iteration:{}. Loss:{}. Accuracy:{}'.format(iter,loss.data[0],accuracy))

RuntimeError: $ Torch: not enough memory: you tried to allocate 2GB. Buy new RAM! at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/TH/THGeneral.cpp:204